In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import WebDriverException, JavascriptException, \
NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
#import getpass
import pandas as pd
import pickle
import os
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
#from time import sleep
import re
import time
from tqdm import tqdm
import winsound
import traceback


In [2]:
# Ouverture du driver et login a scopus

identifiant = '****'
mot_de_passe = '****'

def open_scopus(id = identifiant, mdp = mot_de_passe) :
    
    sleep_max = 2
    
    drv = webdriver.Chrome(ChromeDriverManager().install())
    drv.get("https://www.scopus.com/search/form.uri?display=advanced")
    drv.maximize_window()

    searched_elem = '//*[@id="signin_link_move"]'
    WebDriverWait(drv, sleep_max).until(lambda drv: drv.find_elements_by_xpath(searched_elem))
    drv.find_element_by_xpath(searched_elem).click()
    searched_elem = '//*[@class="els-secondaryBtn btn-border signInInstBtn"]'
    WebDriverWait(drv, sleep_max).until(lambda drv: drv.find_elements_by_xpath(searched_elem))
    drv.find_element_by_xpath(searched_elem).click()
    searched_elem = '/html/body/div/section/main/div[2]/div[2]/div/form/div[1]/input'
    WebDriverWait(drv, sleep_max).until(lambda drv: drv.find_elements_by_xpath(searched_elem))
    drv.find_element_by_xpath(searched_elem).send_keys('ENSAE')
    searched_elem = '/html/body/div/section/main/div[2]/div[2]/div/form/div[2]/div/div/form[3]/button'
    WebDriverWait(drv, sleep_max).until(lambda drv: drv.find_elements_by_xpath(searched_elem))
    drv.find_element_by_xpath(searched_elem).click()
    searched_elem = '/html/body/div/section/main/form/div[4]/div[4]/button/div/div[3]'
    WebDriverWait(drv, sleep_max).until(lambda drv: drv.find_elements_by_xpath(searched_elem))
    drv.find_element_by_xpath(searched_elem).click()
    searched_elem = '/html/body/div/div[2]/form/table/tbody/tr[1]/td[3]/input'
    WebDriverWait(drv, sleep_max).until(lambda drv: drv.find_elements_by_xpath(searched_elem))
    drv.find_element_by_xpath(searched_elem).send_keys(id)
    searched_elem = '/html/body/div/div[2]/form/table/tbody/tr[2]/td[2]/input'
    WebDriverWait(drv, sleep_max).until(lambda drv: drv.find_elements_by_xpath(searched_elem))
    drv.find_element_by_xpath(searched_elem).send_keys(mdp)
    searched_elem = '/html/body/div/div[2]/form/table/tbody/tr[3]/td[3]/button'
    WebDriverWait(drv, sleep_max).until(lambda drv: drv.find_elements_by_xpath(searched_elem))
    drv.find_element_by_xpath(searched_elem).click()
    
    return(drv)

# Défintion de la fonction de scraping

In [27]:
def fonction_scraping(base_req, pth, trkr_nm) :
    
    os.chdir(pth)
    sleep_max = 2

    try :
        for i in tqdm(range(0,len(base_req['req_tot'])), position=0, leave=True) :
            BOOL = (base_req.loc[i, 'done'] == 0) and (base_req.loc[i, 'problematic'] == 0)
            if BOOL :
                query = base_req.loc[i, 'req_tot']

                try :
                    searched_elem = '/html/body/div[1]/div[1]/div/div[2]/div[1]/div[3]/div[1]/div[2]/div/form/div/div[1]/div/div/div[2]/div/section/div[2]/ul/li[4]/a'
                    WebDriverWait(driver, sleep_max).until(lambda driver: driver.find_elements_by_xpath(searched_elem))
                    driver.find_element_by_xpath(searched_elem).click()
                except :
                    pass
                searched_elem = '/html/body/div[1]/div[1]/div/div[2]/div[1]/div[3]/div[1]/div[2]/div/form/div/div[1]/div/div/div[2]/div/section/div[1]/div[1]/label'
                WebDriverWait(driver, sleep_max).until(lambda driver: driver.find_elements_by_xpath(searched_elem))
                driver.find_element_by_xpath(searched_elem).click()
                time.sleep(2)
                searched_elem = '/html/body/div[1]/div[1]/div/div[2]/div[1]/div[3]/div[1]/div[2]/div/form/div/div[1]/div/div/div[2]/div/section/div[1]/div[1]/div'
                WebDriverWait(driver, sleep_max).until(lambda driver: driver.find_elements_by_xpath(searched_elem))
                driver.find_element_by_xpath(searched_elem).send_keys(query)
                searched_elem = '/html/body/div[1]/div/div[1]/div[2]/div/div[3]/div/div[2]/div/form/div/div[1]/div/div/div[2]/div/section/div[2]/ul/li[5]/button/span[1]'
                WebDriverWait(driver, sleep_max).until(lambda driver: driver.find_elements_by_xpath(searched_elem))
                driver.find_element_by_xpath(searched_elem).click()
                try :
                    searched_elem = '/html/body/div/div/div[1]/div/div/div[3]/div/form/div[2]/div/header/h1'
                    WebDriverWait(driver, sleep_max).until(lambda driver: driver.find_elements_by_xpath(searched_elem))
                    driver.find_element_by_xpath(searched_elem).click()
                except :
                    searched_elem = '/html/body/div[1]/div/div[1]/div/div/div[3]/form/div[1]/div/header/h1/span[1]'
                    WebDriverWait(driver, sleep_max).until(lambda driver: driver.find_elements_by_xpath(searched_elem))
                    nb_docs = driver.find_element_by_xpath(searched_elem).get_attribute("innerHTML")
                    nb_docs = re.sub("[^0-9]", "", nb_docs)
                    nb_docs = int(nb_docs)
                    if nb_docs > 1999 :
                        print("Plus de 2000 articles")
                        base_req.loc[i, 'problematic'] = 1
                    else :
                        searched_elem = '//*[@class="ico-navigate-down icon-after fontSizeNorm"]'
                        WebDriverWait(driver, sleep_max).until(lambda driver: driver.find_elements_by_xpath(searched_elem))
                        driver.find_element_by_xpath(searched_elem).click()
                        searched_elem = '//*[@for="mainResults-selectAllTop"]'
                        WebDriverWait(driver, sleep_max).until(lambda driver: driver.find_elements_by_xpath(searched_elem))
                        driver.find_element_by_xpath(searched_elem).click()               
                        searched_elem = '//*[@id="directExport"]'
                        WebDriverWait(driver, sleep_max).until(lambda driver: driver.find_elements_by_xpath(searched_elem))
                        driver.find_element_by_xpath(searched_elem).click()
                searched_elem = '//*[@class="ico-pencil"]'
                WebDriverWait(driver, sleep_max).until(lambda driver: driver.find_elements_by_xpath(searched_elem))
                driver.find_element_by_xpath(searched_elem).click()
                searched_elem = '/html/body/div[1]/div[1]/div/div[2]/div[1]/div[3]/div[1]/div[2]/div/form/div/div[1]/div/div/div[2]/div/section/div[2]/ul/li[4]/a'
                WebDriverWait(driver, sleep_max).until(lambda driver: driver.find_elements_by_xpath(searched_elem))
                driver.find_element_by_xpath(searched_elem).click()
                base_req.loc[i, 'done'] = 1
            
            base_req.to_csv(trkr_nm)
            
    except :
        pass
        
    return(base_req)


# Scraping articles auteurs

In [28]:
driver = open_scopus()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST driver version for 100.0.4896
Driver [C:\Users\Dell\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [29]:
path = 'C:\\Users\\Dell\\Desktop\\DSSS propre\\Queries'
os.chdir(path)
base_requetes = pd.read_csv('base_requetes_auteurs.csv')
base_requetes_1 = base_requetes[base_requetes['repartition_requetes'] == 'Maxime1']
base_requetes_1 = base_requetes_1.reset_index()
tracker_name = 'query_tracker_1'

query_tracker_1 = fonction_scraping(base_req = base_requetes_1, pth = path, trkr_nm = tracker_name)

  0%|▎                                                                               | 1/228 [00:29<1:51:54, 29.58s/it]


In [26]:
path = 'C:\\Users\\Dell\\Desktop\\DSSS propre\\Queries'
os.chdir(path)
base_requetes = pd.read_csv('base_requetes_auteurs.csv')
base_requetes_2 = base_requetes[base_requetes['repartition_requetes'] == 'Maxime2']
base_requetes_2 = base_requetes_2.reset_index()
tracker_name = 'query_tracker_2'

query_tracker_2 = fonction_scraping(base_req = base_requetes_2, pth = path, trkr_nm = tracker_name)

  0%|                                                                                          | 0/227 [00:00<?, ?it/s]

     index  Unnamed: 0                                            req_tot  \
0      454         455  AU-ID(57208338872) OR AU-ID(57208340124) OR AU...   
1      455         456  AU-ID(57208482840) OR AU-ID(57208515161) OR AU...   
2      456         457  AU-ID(57208706416) OR AU-ID(57208731140) OR AU...   
3      457         458  AU-ID(57208844019) OR AU-ID(57208845001) OR AU...   
4      458         459  AU-ID(57209053280) OR AU-ID(57209056563) OR AU...   
..     ...         ...                                                ...   
222    676         677  AU-ID(57271434900) OR AU-ID(57272427300) OR AU...   
223    677         678  AU-ID(57282561200) OR AU-ID(57282939600) OR AU...   
224    678         679  AU-ID(57290803700) OR AU-ID(57290833300) OR AU...   
225    679         680  AU-ID(57299627800) OR AU-ID(57299919700) OR AU...   
226    680         681  AU-ID(57313054200) OR AU-ID(57313964700) OR AU...   

    repartition_requetes  done  problematic  
0                Maxime2     

  0%|                                                                                          | 0/227 [00:27<?, ?it/s]
